<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Embedding-training-(triplet-loss)" data-toc-modified-id="Embedding-training-(triplet-loss)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Embedding training (triplet loss)</a></span></li><li><span><a href="#Binary-classifier" data-toc-modified-id="Binary-classifier-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Binary classifier</a></span></li></ul></div>

In [2]:
import os
import random
import functools
from functools import partial
import PIL

import numpy as np 
import pandas as pd

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import timm

from transformers import AutoTokenizer
from transformers import DistilBertModel

np.random.seed(1337)

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
# train val split
labels = np.random.permutation(df['label_group'].unique())

train_perc = 0.7
train_idx = int(train_perc * len(labels))

train_labels = labels[:train_idx]
val_labels = labels[train_idx:]

train_df = df[df['label_group'].isin(train_labels)]
val_df = df[df['label_group'].isin(val_labels)]

In [4]:

images_path = 'data/train_images/'
image_ids = [s.split('.')[0] for s in os.listdir(images_path)]
image_ids[:10]

['0000a68812bc7e98c42888dfb1c07da0',
 '00039780dfc94d01db8676fe789ecd05',
 '000a190fdd715a2a36faed16e2c65df7',
 '00117e4fc239b1b641ff08340b429633',
 '00136d1cf4edede0203f32f05f660588',
 '0013e7355ffc5ff8fb1ccad3e42d92fe',
 '00144a49c56599d45354a1c28104c039',
 '0014f61389cbaa687a58e38a97b6383d',
 '0019a3c6755a194cb2e2c12bfc63972e',
 '001be52b2beec40ddc1d2d7fc7a68f08']

## Embedding training (triplet loss)

In [ ]:
class TripletDataset(Dataset) :
    def __init__(self, images_path, df, img_tfms, testing, text_tokenizer=None):
        super(TripletDataset, self).__init__()
        
        self.images_path = images_path
        self.img_tfms = img_tfms
        self.testing = testing
              
        self.df = df.copy()
        self.df['label_group'] = self.df['label_group'].astype('category').cat.codes
        self.df['index'] = range(self.df.shape[0])
        self.labels = self.df['label_group'].unique()
        self.label_to_index_list = self.df.groupby('label_group')['index'].apply(list)
        
    def __getitem__(self, index) :
        index_meta = self.df.iloc[index]
        
        anchor_image, anchor_text = self._get_item(index)
        
        if self.testing: return anchor_image, anchor_text
        
        label = index_meta['label_group']
        
        # positive sample
        pos_index = random.choice(self.label_to_index_list[label])
        # we don't want the positive sample being the same as the anchor
        while pos_index == index :
            pos_index = random.choice(self.label_to_index_list[label])
        pos_image, pos_text = self._get_item(pos_index)
        
        #negative sample
        neg_label = random.choice(self.labels)
        # Negative sample has to be different label from anchor 
        while neg_label == index :
            neg_label = random.choice(self.labels)
        neg_index = random.choice(self.label_to_index_list[neg_label])
        neg_image, neg_text = self._get_item(neg_index)
        
        return anchor_image, anchor_text, pos_image, pos_text, neg_image, neg_text
        
    def _get_item(self, index) :
        image = PIL.Image.open(os.path.join(self.images_path, 
                                            self.df.iloc[index]['image']))
        image = self.img_tfms(image)
        text = self.df.iloc[index]['title']
        return image, text
    
    def __len__(self) :
        return self.df.shape[0]

In [7]:
def create_dl(images_path, df_paths, img_tfms, pretrianed_tokenizer='distilbert-base-uncased', 
              batch_size=64, shuffle = True, testing = False) :
    dataset = TripletDataset(images_path, df_paths, img_tfms, testing)
    tokenizer = AutoTokenizer.from_pretrained(pretrianed_tokenizer)
    dl = DataLoader(dataset, batch_size=batch_size, collate_fn=partial(collate_fn, tokenizer), 
                    shuffle = shuffle, pin_memory = True)
    return dl

In [8]:

class EmbedorNN(nn.Module) :
    def __init__(self, pretrained_image_embedor='resnet18', pretrained_text_embedor='distilbert-base-uncased',
                output_dim=128) :
        super(EmbedorNN, self).__init__()
        self.image_embedor = timm.create_model(pretrained_image_embedor, pretrained=True)
        self.image_pool = nn.AdaptiveAvgPool2d((1,1))
        self.text_embedor = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.head = nn.Linear(512+768, output_dim)
    
    def forward(self, x) :
        images, texts = x
        out_images = self.image_embedor.forward_features(images)
        out_images = self.image_pool(out_images).squeeze()
        out_text = self.text_embedor(texts['input_ids'], 
                                     attention_mask=texts['attention_mask'])[0][:,0,:]
        out = torch.cat([out_images, out_text], dim=-1)
        return self.head(out)

In [9]:
normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.Resize((250, 250)),
                                       transforms.ColorJitter(.25,.25,.25),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     normalize
                                     ])

In [10]:
device = torch.device('cuda')

In [23]:
model = EmbedorNN().to(device)
# weights: https://drive.google.com/drive/folders/19BGTC53p5YIAakWeCZfNIsceMX7cjHgp?usp=sharing
model.load_state_dict(torch.load('3ep.pth'))

<All keys matched successfully>

In [12]:
tr_dl = create_dl(images_path, train_df, train_transforms, shuffle = True)

In [13]:
val_dl = create_dl(images_path, val_df, val_transforms, shuffle = False)

In [13]:
testing_dl = create_dl(images_path, df, val_transforms, shuffle = False, testing = True)

In [14]:
n_epochs = 3
swa_start = int(0.75*n_epochs)

lf = nn.TripletMarginLoss()

lr = 1e-4
wd = 1e-3
no_decay = ["bias", "BatchNorm2d.weight", "BatchNorm2d.bias", "LayerNorm.weight", 'LayerNorm.bias']

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": wd,
    },
    {
        "params": [p for n, p in  model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=lr)

# learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr =lr, pct_start = 0.3, #anneal_strategy = 'linear',
                                            total_steps = int(n_epochs * len(tr_dl)))

scaler = torch.cuda.amp.GradScaler()

In [15]:
tr_losses = []
val_losses = []
for ep in tqdm(range(n_epochs)):
    model.train()
    tr_loss = []
    pbar = tqdm(tr_dl)
    for anchor_image, anchor_text, pos_image, pos_text, neg_image, neg_text in pbar:
        
        
        anchor = anchor_image.to(device), {'input_ids' : anchor_text['input_ids'].to(device),
                                           'attention_mask' : anchor_text['attention_mask'].to(device)}
        
        pos = pos_image.to(device), {'input_ids' : pos_text['input_ids'].to(device),
                                     'attention_mask' : pos_text['attention_mask'].to(device)}
        
        neg = neg_image.to(device), {'input_ids' : neg_text['input_ids'].to(device),
                                     'attention_mask' : neg_text['attention_mask'].to(device)}
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            anchor_emb = model(anchor)
            pos_emb = model(pos)
            neg_emb = model(neg)
            loss = lf(anchor_emb, pos_emb, neg_emb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        sched.step()
        
        tr_loss.append(loss.item())
        pbar.set_description(f"Train loss: {round(np.mean(tr_loss),3)}")
        
    model.eval()
    val_loss = []
    with torch.no_grad():
        pbar = tqdm(val_dl)
        for anchor_image, anchor_text, pos_image, pos_text, neg_image, neg_text in pbar:

            anchor = anchor_image.to(device), {'input_ids' : anchor_text['input_ids'].to(device),
                                               'attention_mask' : anchor_text['attention_mask'].to(device)}

            pos = pos_image.to(device), {'input_ids' : pos_text['input_ids'].to(device),
                                         'attention_mask' : pos_text['attention_mask'].to(device)}

            neg = neg_image.to(device), {'input_ids' : neg_text['input_ids'].to(device),
                                         'attention_mask' : neg_text['attention_mask'].to(device)}

            with torch.cuda.amp.autocast():
                
                anchor_emb = model(anchor)
                pos_emb = model(pos)
                neg_emb = model(neg)
                loss = lf(anchor_emb, pos_emb, neg_emb)

            val_loss.append(loss.item())
            pbar.set_description(f"Val loss: {round(np.mean(val_loss),3)}")
            
    tr_loss = round(np.mean(tr_loss),3)
    val_loss = round(np.mean(val_loss),3)
    tr_losses.append(tr_loss)
    val_losses.append(val_loss)
    summary = f"Ep {ep}: Train loss {tr_loss} - Val loss {val_loss}"
    print(summary) 
    
    

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

Ep 0: Train loss 0.064 - Val loss 0.023


  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

Ep 1: Train loss 0.018 - Val loss 0.018


  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?it/s]

Ep 2: Train loss 0.008 - Val loss 0.012


In [16]:
torch.save(model.state_dict(), '3ep.pth')

In [15]:
embs = []
model.eval()
with torch.no_grad():
    pbar = tqdm(testing_dl)
    for image, text in pbar:
        x = image.to(device), {'input_ids' : text['input_ids'].to(device),
                               'attention_mask' : text['attention_mask'].to(device)}
        y = model(x)
        embs.append(y.cpu())

  0%|          | 0/536 [00:00<?, ?it/s]

In [18]:
embs = torch.cat(embs,0)

In [19]:
embs_df = pd.DataFrame(embs.numpy())
emb_cols = [f'emb_{i}' for i in embs_df.columns]
embs_df.columns = emb_cols
embs_df['cls'] = df['label_group']
embs_df['cls'] = embs_df['cls'].astype('category').cat.codes

In [21]:
embs_df.T

,0,1,2,3,4,5,6,7,8,9,...,34240,34241,34242,34243,34244,34245,34246,34247,34248,34249
emb_0,0.500960,0.237719,0.050669,0.718747,-0.581001,-0.097971,1.183702,1.779962,1.124665,-0.972785,...,-1.564198,0.842688,-0.393273,-0.365358,-0.153456,-0.877197,0.058753,-0.112191,0.031476,0.074278
emb_1,-1.301394,-1.292176,-0.944553,-1.888656,-0.108013,-0.747545,-1.263595,-1.211068,-0.682983,0.179870,...,-0.576654,-0.905915,-0.449244,-0.466996,-0.804786,-0.308678,0.259296,-0.835542,-0.930508,-1.531453
emb_2,-0.226438,0.306056,0.002641,0.311137,-0.544753,-0.013903,0.166399,0.306137,0.915265,-0.525376,...,0.455988,0.210848,-0.447949,-0.411966,0.376221,-0.085963,0.045716,0.809573,-0.265023,0.909191
emb_3,-0.323761,0.608568,-0.098363,-1.106508,0.221210,-1.320947,-0.539112,-1.217819,0.357320,-0.854563,...,0.597615,-0.977988,1.731781,1.720604,0.479843,0.006830,0.526009,-0.228569,0.531991,0.439010
emb_4,-1.258861,-0.940595,0.709853,-0.407680,0.241497,-1.016383,-0.178965,-0.405246,0.401834,-0.620351,...,-0.869939,-1.265399,-0.175623,-0.220973,-0.101543,-1.238433,-0.110854,-0.539160,0.548752,-1.179566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
emb_124,1.327470,0.503480,0.680511,0.594233,-0.113412,-0.175654,0.622813,0.947124,0.934831,-0.190990,...,-0.169104,0.302227,1.240302,1.217515,-0.716665,-0.396859,0.584340,-0.777377,0.648708,-0.493271
emb_125,-1.127460,-0.561340,0.595688,-0.708277,-0.253344,-1.247321,-0.849796,-1.481531,0.011094,-1.126404,...,0.344415,-0.091109,1.479870,1.414771,-0.354530,0.216521,-0.839207,-1.230204,0.316971,-1.293721
emb_126,0.465477,-1.320146,0.288588,0.464027,0.089630,-0.426252,-0.943907,0.281021,0.681009,0.782818,...,1.783656,0.248377,0.240647,0.235544,0.519699,-0.012075,0.809881,1.609108,-0.025901,-0.320838
emb_127,1.034373,-0.168213,-0.304716,-0.008191,0.086427,1.383384,0.928859,0.351101,0.364785,0.460414,...,-0.302482,-0.573974,0.280889,0.266239,-0.264183,-0.374306,1.208320,-0.497938,-0.462315,0.550078


In [22]:
embs_df.to_csv('train_embs.csv')

## Binary classifier 

In [1]:
import os
import random
import functools
from functools import partial
import PIL

import numpy as np 
import pandas as pd

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import timm

from transformers import AutoTokenizer
from transformers import DistilBertModel

np.random.seed(1337)

In [2]:
from models import EmbedorNN, Decidor
from data_process import collate_fn, TripletDataset, create_dl

In [3]:
device = 'cuda'

In [4]:
model = EmbedorNN().to(device)
# weights: https://drive.google.com/drive/folders/19BGTC53p5YIAakWeCZfNIsceMX7cjHgp?usp=sharing
model.load_state_dict(torch.load('data/3ep.pth'))

<All keys matched successfully>

In [5]:
df = pd.read_csv('data/train.csv')

In [6]:
images_path = 'data/train_images'

In [7]:
# train val split
labels = np.random.permutation(df['label_group'].unique())

train_perc = 0.7
train_idx = int(train_perc * len(labels))

train_labels = labels[:train_idx]
val_labels = labels[train_idx:]

train_df = df[df['label_group'].isin(train_labels)]
val_df = df[df['label_group'].isin(val_labels)]

In [8]:
normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                 std=(0.229, 0.224, 0.225))

train_transforms = transforms.Compose([transforms.Resize((250, 250)),
                                       transforms.ColorJitter(.25,.25,.25),
                                       transforms.RandomRotation(5),
                                       transforms.RandomCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       normalize
                                       ])

val_transforms = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     normalize
                                     ])

In [9]:
tr_dl = create_dl(images_path, train_df, train_transforms, shuffle = True)

val_dl = create_dl(images_path, val_df, val_transforms, shuffle = False)

testing_dl = create_dl(images_path, df, val_transforms, shuffle = False, testing = True)

In [ ]:
embs = []
model.eval()
with torch.no_grad():
    pbar = tqdm(testing_dl)
    for image, text in pbar:
        x = image.to(device), {'input_ids' : text['input_ids'].to(device),
                               'attention_mask' : text['attention_mask'].to(device)}
        y = model(x)
        embs.append(y.cpu())

In [10]:
df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [12]:
embs = torch.cat(embs,0)

NameError: name 'embs' is not defined

In [25]:
embs.shape

torch.Size([34250, 128])

In [179]:
#torch.save(embs, 'embs.pkl')

In [11]:
#embs = torch.load('embs.pkl')

In [88]:
class BinaryClassDS(Dataset) :
    def __init__(self, embeddings, df, testing):
        super(BinaryClassDS, self).__init__()
        self.embeddings = embeddings
        
        self.testing = testing
              
        self.df = df.copy()
        self.df['label_group'] = self.df['label_group'].astype('category').cat.codes
        self.df['index'] = range(self.df.shape[0])
        self.labels = self.df['label_group'].unique()
        self.label_to_index_list = self.df.groupby('label_group')['index'].apply(list)
        
    def __getitem__(self, index) :
        index_meta = self.df.iloc[index]
        
        anchor_emb = self.embeddings[index]
            
        label = index_meta['label_group']
        
        # positive sample
        pos_index = random.choice(self.label_to_index_list[label])
        # we don't want the positive sample being the same as the anchor
        while pos_index == index :
            pos_index = random.choice(self.label_to_index_list[label])
        pos_emb = self.embeddings[pos_index]
        
        #negative sample
        neg_label = random.choice(self.labels)
        # Negative sample has to be different label from anchor 
        while neg_label == index :
            neg_label = random.choice(self.labels)
        neg_index = random.choice(self.label_to_index_list[neg_label])
        neg_emb = self.embeddings[neg_index]
        x = torch.stack([torch.cat([anchor_emb, pos_emb], dim=-1), 
                        torch.cat([anchor_emb, neg_emb], dim=-1)])
        distances = [torch.dist(anchor_emb, pos_emb), torch.dist(anchor_emb, neg_emb)]
        x = torch.cat([x, torch.stack(distances)[:, None]], dim=-1)
        y = torch.tensor([1, 0])
        
        return x, y
    
    def __len__(self) :
        return self.df.shape[0]

In [89]:
def collate_fn_binary(samples) :
    x, y = zip(*samples)
    x = torch.cat(x, dim=0)
    y = torch.cat(y, dim=0)
    return x, y

In [90]:
train_dl = DataLoader(BinaryClassDS(embs[list(train_df.index)], train_df, False), batch_size=124, shuffle=True, collate_fn=collate_fn_binary)
val_dl = DataLoader(BinaryClassDS(embs[list(val_df.index)], val_df, False), batch_size=124, shuffle=False, collate_fn=collate_fn_binary)

In [91]:
x, y = next(iter(train_dl))

In [92]:
x.shape, y.shape

(torch.Size([248, 257]), torch.Size([248]))

In [93]:
_layers = []
_layers.append(nn.Linear(128*2+1, 10))
_layers.append(nn.BatchNorm1d(10))
_layers.append(nn.ReLU())
_layers.append(nn.Dropout(0.1))
_layers.append(nn.Linear(10, 1))
decidor = nn.Sequential(*_layers)

In [94]:
n_epochs = 10
swa_start = int(0.75*n_epochs)

s = nn.Sigmoid()
lf = nn.BCELoss()

lr = 1e-2
wd = 0

optimizer = torch.optim.AdamW(decidor.parameters(), lr=lr)

# learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr =lr, pct_start = 0.3, #anneal_strategy = 'linear',
                                            total_steps = int(n_epochs * len(train_dl)))


tr_losses = []
val_losses = []
for ep in tqdm(range(n_epochs)):
    decidor.train()
    tr_loss = []
    pbar = tqdm(train_dl)
    for x, y in pbar:
        
        optimizer.zero_grad()
        y_pred = s(decidor(x)).squeeze() 
        loss = lf(y_pred, y.float())
        loss.backward()
        optimizer.step()
        sched.step()
        
        tr_loss.append(loss.item())
        pbar.set_description(f"Train loss: {round(np.mean(tr_loss),3)}")
        
    decidor.eval()
    val_loss = []
    with torch.no_grad():
        pbar = tqdm(val_dl)
        for x, y in pbar:
            y_pred = s(decidor(x)).squeeze()
            loss = lf(y_pred, y.float())

            val_loss.append(loss.item())
            pbar.set_description(f"Val loss: {round(np.mean(val_loss),3)}")
            
    tr_loss = round(np.mean(tr_loss),3)
    val_loss = round(np.mean(val_loss),3)
    tr_losses.append(tr_loss)
    val_losses.append(val_loss)
    summary = f"Ep {ep}: Train loss {tr_loss} - Val loss {val_loss}"
    print(summary) 
    
    


Ep 0: Train loss 0.491 - Val loss 0.193



Ep 1: Train loss 0.1 - Val loss 0.118



Ep 2: Train loss 0.067 - Val loss 0.107



Ep 3: Train loss 0.064 - Val loss 0.086



Ep 4: Train loss 0.058 - Val loss 0.086



Ep 5: Train loss 0.056 - Val loss 0.085



Ep 6: Train loss 0.055 - Val loss 0.085



Ep 7: Train loss 0.054 - Val loss 0.084



Ep 8: Train loss 0.051 - Val loss 0.084



Ep 9: Train loss 0.049 - Val loss 0.085



In [97]:
with torch.no_grad():
    acc = 0
    pbar = tqdm(val_dl)
    for x, y in pbar:
        y_pred = s(decidor(x)).squeeze() > 0.5
        acc += (y_pred==y).float().mean()
acc/len(val_dl)
            

tensor(0.9735)

In [100]:
torch.save(decidor.state_dict(), 'data/3ep_decidor.pth')